In [82]:
import requests
import json
import random

if False:
    BADGERDOC_HOST = "http://13.92.243.227:8080"
    KEYCLOAK_TOKEN_URI = (
        "http://13.92.243.227:8082/auth/realms/master/protocol/openid-connect/token/"
    )
    KEYCLOAK_SYSTEM_USER_SECRET = "***"
else:
    BADGERDOC_HOST = "http://demo.badgerdoc.com:8080"
    KEYCLOAK_TOKEN_URI = "http://demo.badgerdoc.com:8082/auth/realms/master/protocol/openid-connect/token/"
    KEYCLOAK_SYSTEM_USER_SECRET = "***"

BADGERDOC_TENANT = "badgerdoc"
BADGERDOC_JOB_ID = 104
BADGERDOC_FILE_ID = 360
EXTERNAL_PIPELINE_ID = 0

KEYCLOAK_SYSTEM_USER_CLIENT = "badgerdoc-internal"

In [90]:
def gpt_to_badgerdoc_annotation(gpt_key, gpt_value):
    default_str = "unknown"
    if gpt_key == "patient_information":
        age = gpt_value.get("age", default_str)
        age_unit = gpt_value.get("age_unit", "age_unit")
        gender = gpt_value.get("gender", default_str)
        ethnic_group = gpt_value.get("ethnic_group", default_str)
        yield (
            gpt_key,
            f"Age: {age} {age_unit}\nGender: {gender}\nEthnic group: {ethnic_group}",
        )
        yield ("age", age)
    elif gpt_key == "patient_examination":
        for examination in gpt_value:
            exam_text = examination.get("exam_text", default_str)
            yield (gpt_key, f"{exam_text}")
    elif gpt_key == "initial_conidition":
        for condition in gpt_value:
            yield (
                "reported_term_local",
                condition.get("reported_term_local")
            )


def extract_objs(gpt):
    bd_objs = []
    for needle in gpt:
        for gpt_key, gpt_value in needle.items():
            for category, bd_value in gpt_to_badgerdoc_annotation(gpt_key, gpt_value):
                if category:
                    bd_objs.append(
                        {
                            "id": random.choice(range(0, 0xFFFFFF)),
                            "category": category,
                            "type": "document",
                            "text": bd_value,
                        }
                    )

    return bd_objs


def get_service_token(client, secret):
    """Creates Pipelines client token"""
    url = KEYCLOAK_TOKEN_URI
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    payload = {
        "client_id": client,
        "client_secret": secret,
        "grant_type": "client_credentials",
    }
    response = requests.request(method="POST", url=url, data=payload, headers=headers)
    response.raise_for_status()
    try:
        return response.json()["access_token"]
    except KeyError:
        print(f"{response=}, {response.json()=}")
    return None


def commit_annotation(token, body, job_id, file_id):
    annotation_url = f"{BADGERDOC_HOST}/annotation/annotation/{job_id}/{file_id}"
    headers = {"X-Current-Tenant": BADGERDOC_TENANT, "Authorization": f"Bearer {token}"}
    return requests.post(url=annotation_url, headers=headers, json=body)


def start_job(token, tenant, job_id):
    start_job_url = f"{BADGERDOC_HOST}/annotation/jobs/{job_id}/start"
    headers = {"X-Current-Tenant": tenant, "Authorization": f"Bearer {token}"}
    response = requests.post(url=start_job_url, headers=headers)
    response.raise_for_status()
    return response

# Form badgerdoc commit body from `gpt_output_example.json` file

In [91]:
with open("gpt_output_example.json", "r") as output_file:
    output_gpt = json.load(output_file)

badgerdoc_commit_body = {
    "base_revision": None,
    "user": None,
    "pipeline": EXTERNAL_PIPELINE_ID,
    "pages": [
        {
            "page_num": 1,
            "size": {
                "width": 0,
                "height": 0,
            },
            "objs": extract_objs(output_gpt)
        }
    ],
}
print(badgerdoc_commit_body)

{'base_revision': None, 'user': None, 'pipeline': 0, 'pages': [{'page_num': 1, 'size': {'width': 0, 'height': 0}, 'objs': [{'id': 11132028, 'category': 'patient_information', 'type': 'document', 'text': 'Age: 75 year\nGender: female\nEthnic group: Japanese'}, {'id': 6659546, 'category': 'age', 'type': 'document', 'text': 75}, {'id': 2749430, 'category': 'patient_examination', 'type': 'document', 'text': 'A 75-year-old Japanese female with a history of diabetes mellitus, smoking, and PAD, underwent lower-extremity peripheral vascular intervention using Fowler’s position and the Terumo R2P system for treatment of right SFA chronic total occlusion. Post-procedure, her ABI improved to 0.78 in the right leg, and she was discharged 1 week after PVI. At 6 months post-PVI, she was free from pain in her right lower leg with an ABI of 0.72.'}, {'id': 394447, 'category': 'patient_information', 'type': 'document', 'text': 'Age: 74 year\nGender: male\nEthnic group: Japanese'}, {'id': 9790306, 'cate

# Commit annotation

In [92]:

token = get_service_token(KEYCLOAK_SYSTEM_USER_CLIENT, KEYCLOAK_SYSTEM_USER_SECRET)
if token is not None:
    commit_response = commit_annotation(token, badgerdoc_commit_body, BADGERDOC_JOB_ID, BADGERDOC_FILE_ID)
    print(f"{commit_response=}, {commit_response.json()=}")


commit_response=<Response [201]>, commit_response.json()={'revision': '8f1203bcab99252d59b77255682a3c5ffc6a0d74', 'user': None, 'pipeline': 0, 'date': '2024-05-28T13:46:20.700467', 'file_id': 360, 'job_id': 104, 'pages': {'1': '16df0a1daff5f83c078c123513e579b96a033177'}, 'validated': [], 'failed_validation_pages': [], 'tenant': 'badgerdoc', 'task_id': None, 'similar_revisions': None, 'categories': [], 'links_json': None}


# Start job

In [93]:
if token is not None:
    start_job_response = start_job(token, BADGERDOC_TENANT, BADGERDOC_JOB_ID)
    print(f"{start_job_response=}, {start_job_response.json()=}")

start_job_response=<Response [200]>, start_job_response.json()=[{'id': 204, 'status': 'Ready', 'file_id': 361, 'pages': [1, 2, 3, 4, 5, 6], 'job_id': 104, 'user_id': '0dc326e4-b190-4881-8d05-12359052abbf', 'is_validation': True, 'deadline': None}, {'id': 205, 'status': 'Ready', 'file_id': 360, 'pages': [1, 2, 3, 4, 5, 6], 'job_id': 104, 'user_id': '0dc326e4-b190-4881-8d05-12359052abbf', 'is_validation': True, 'deadline': None}]


In [95]:
def get_revision(token, tenant, job_id, file_id, revision):
    revision_url = f"{BADGERDOC_HOST}/annotation/annotation/{job_id}/{file_id}/{revision}?page_numbers=1"
    headers = {"X-Current-Tenant": tenant, "Authorization": f"Bearer {token}"}
    response = requests.get(url=revision_url, headers=headers)
    response.raise_for_status()
    return response

if token is not None:
    revision_response = get_revision(token, BADGERDOC_TENANT, BADGERDOC_JOB_ID, BADGERDOC_FILE_ID, "41bd530ef56e676e9a72a1c4527c3793b397d4c1")
    print(revision_response.json())
    

{'revision': '41bd530ef56e676e9a72a1c4527c3793b397d4c1', 'user': '0dc326e4-b190-4881-8d05-12359052abbf', 'pipeline': None, 'date': '2024-05-28T14:06:50.952070', 'pages': [{'page_num': 1, 'size': {'width': 595.276, 'height': 790.866}, 'objs': [{'id': 11132028, 'type': 'document', 'bbox': [0, 0, 0, 0], 'category': 'patient_information', 'data': {'dataAttributes': []}, 'children': [6659546, 2749430, 394447, 9790306, 6692901], 'text': 'Age: 75 year\nGender: female\nEthnic group: Japanese'}, {'id': 6659546, 'type': 'document', 'bbox': [0, 0, 0, 0], 'category': 'age', 'data': {'dataAttributes': []}, 'children': [11132028, 2749430, 394447, 9790306, 6692901], 'text': 75}, {'id': 2749430, 'type': 'document', 'bbox': [0, 0, 0, 0], 'category': 'patient_examination', 'data': {'dataAttributes': []}, 'children': [11132028, 6659546, 394447, 9790306, 6692901], 'text': 'A 75-year-old Japanese female with a history of diabetes mellitus, smoking, and PAD, underwent lower-extremity peripheral vascular int